In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 7.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595743 sha256=d53fc44dd2ee60f7341e1f6f76d50c304d907b5b6f9ffdc05f37bdd779ad1daa
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.1 MB/s 
     |████████████████████████████████| 769 kB 8.3 MB/s 
     |████████████████████████████████| 3.0 MB 57.0 MB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-po0ahj72
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-po0ahj72
     |████████████████████████████████| 131 kB 9.7 MB/s 
     |████████████████████████████████| 4.5 MB 37.9 MB/s 
     |████████████████████████████████| 3.4 MB 65.2 MB/s 
     |████████████████████████████████| 3.3 MB 67.7 MB/s 
     |████████████████████████████████| 596 kB 74.2 MB/s 
     |████████████████████████████████| 67 kB 7.8 MB/s 
     |████████████████████████████████| 8.5 MB 66.0 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 127 kB 84.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=f3794729b30f4980c84fc71ac6f490ace0a413980e9fdb2986bc0fdbc93d872a
  Stored in directory: /tmp/pip-ephem-wheel-cache-zxltpogb/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/CUAI Winter/train_v.txt',sep="\t")
valid = pd.read_csv('/content/drive/MyDrive/CUAI Winter/valid_v.txt',sep="\t")

train['sentT'] = train['sent1']+' '+train['sent2']
valid['sentT'] = valid['sent1']+' '+valid['sent2']

train = train[['sentT','sentiment']]
valid = valid[['sentT','sentiment']]

train = train.rename(columns={'sentT':'Sentence','sentiment':'Emotion'})
valid = valid.rename(columns={'sentT':'Sentence','sentiment':'Emotion'})

In [8]:
dataset_train = []
for q, label in zip(train['Sentence'], train['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))
  dataset_train.append(data)

In [9]:
dataset_test = []
for q, label in zip(train['Sentence'], train['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))
  dataset_test.append(data)

In [10]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair
    )
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def  __getitem__(self,i):
    return (self.sentences[i]+(self.labels[i],))
  
  def __len__(self):
    return (len(self.labels))

In [11]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
data_train = BERTDataset(dataset_train , 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test , 0, 1, tok, max_len, True, False)

In [14]:
data_train[0]

(array([   2, 3096, 5330, 1788, 5947, 6855, 4575, 7784, 1767, 6553, 4102,
        3010, 7096, 1370,  517,   54, 3093, 4299, 4102, 3010, 7096, 1370,
         517,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(25, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 6,
               dr_rate = None,
               params = None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_Rate = dr_rate

    self.calssifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout == nn.Dropout(p = dr_rate)
  
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v]=1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    _,pooler = self.bert(input_ids = token_ids, token_type_ids=segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
#model = BERTClassifier(bertmodel)

AttributeError: ignored

In [ ]:
no_decay = ['bias','LayerNorm.weight']
optimizer_grouped_parameters=[
                              {'params': [p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay':0.01
                               },
                              {'params': [p for n,p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader)*num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

In [ ]:
def calc_accuracy(X,Y):
  max_vals, max_indices = torch.max(X,1)
  train_acc = (max_indices==Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long()
    segment_ids = segment_ids.long()
    valid_length = valid_length
    label = label.long()
    out = model(token_ids, valid_length, segment_ids)
    loss = loss_fn(out, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    train_acc += calc_accuracy(out, label)
    if batch_id % log_interval==0:
      print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      print("epoch {} train acc {}".format(e+1, train_Acc / (batch_id+1)))

      model.eval()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length = valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
      print("epoch {} test acc {}".format(e+1, test_acc/(batch_id+1)))